## 2 - Estatística computacional

### 2.1 - Distribuição

- **Distribuição** é um conjunto de valores com as suas probabilidades;
- **Pmf** é a função de massa de probabilidade;

In [1]:
from thinkbayes import Pmf

In [2]:

pmf = Pmf()
for x in [1, 2, 3, 4, 5, 6]:
    pmf.Set(x, 1/6.0)

- **Set** é o método usado para adicionar a probabilidade associada com cada valor;

In [3]:
pmf = Pmf()
word_list = []
for word in word_list:
    pmf.Incr(word, 1)


- **Incr** é o método usada para adicioanar um valor, o valor adicionado não é a probabilidade.
- **Normalize** é o método para calcular a probabilidade(frequência) associado a uma variável.
- **Pmf** é um objeto, podemos perguntar a probabilidade de qualquer valor.
- Pmf usa dicionário para armazernar as valores e as probabilidade;

### 2.2 - O problema do biscoito

- No teorema de bayes, é comum usar Pmf para mapear as probabilidade de cada hipóteses;

No problema do biscoito as hipóteses são B1 e B2;

In [3]:
pmf = Pmf()
pmf.Set('Bowl 1', 0.5)
pmf.Set('Bowl 2', 0.5)

- **Distrubuição prior** é a distribuição que contém a prior de cada hipótese;
- Para atualizar a distribuição baseada em novos dados, multiplica cada priori pelo valor da verrossimilhança correspondente;

In [4]:
pmf.Mult('Bowl 1', 0.75)
pmf.Mult('Bowl 2', 0.5)

- **Mult** multiplica uma dada probabilidade por uma dada verrossimilhança;
- As hipóteses podem ser renomalizada, pois elas são **Multualmente exclusiva** e **Colectivamente exaustiva**.

In [5]:
pmf.Normalize()

0.625

- O resultado é a distribuição posteriori de cada hipótese;

In [6]:
print(pmf.Prob('Bowl 1'))

0.6000000000000001


### 2.3 - O Framework Bayesiano

- Reescreve o código para uma forma mais geral. Define a classe para encapsular os códigos relacionado a cada problema; 

In [2]:
class Cookie(Pmf):
    
    mixes = {
        'Bowl 1': dict(vanilla=0.75, chocolate=0.25),
        'Bowl 2': dict(vanilla=0.5, chocolate=0.5),
    }
    
    def __init__(self, hypos):
        super().__init__(self)
        for hypo in hypos:
            self.Set(hypo, 1)
        self.Normalize()
    
    def Update(self, data):
        for hypo in self.Values():
            like = self.Likelihood(data, hypo)
            self.Mult(hypo, like)
        self.Normalize()
    
    def Likelihood(self, data, hypo):
        mix = self.mixes[hypo]
        like = mix[data]
        return like

- O objeto Cookie é uma Pmf, onde mapeia as hipótese para suas probabilidades; 

In [3]:
hypos = ['Bowl 1', 'Bowl 2']
pmf = Cookie(hypos)
#pmf.Update('vanilla')

- **Update** calcula a verossimilhança e multiplica com a probabilidade da hipótese;

In [26]:
for hypo, prob in pmf.Items():
    print(hypo, prob)

Bowl 1 0.6000000000000001
Bowl 2 0.4


In [4]:
dataset = ['vanilla', 'vanilla', 'vanilla']
for data in dataset:
    pmf.Update(data)
    for hypo, prob in pmf.Items():
        print(hypo, prob)

Bowl 1 0.6000000000000001
Bowl 2 0.4
Bowl 1 0.6923076923076923
Bowl 2 0.30769230769230765
Bowl 1 0.7714285714285714
Bowl 2 0.22857142857142856


- As vantagens de generalizar, são:
    - Podemos extrair mais de um biscoito do pote;
    - Podemos resolver outros problemas semelhantes;

### 2.4 - O problema de Monty Hall

In [9]:
class Monty(Pmf):
    def __init__(self, hypos):
        super().__init__(self)
        for hypo in hypos:
            self.Set(hypo, 1)
        self.Normalize()
    
    def Update(self, data):
        for hypo in self.Values():
            like = self.Likelihood(data, hypo)
            self.Mult(hypo, like)
        self.Normalize()
    
    def Likelihood(self, data, hypo):
        if hypo == data:
            return 0
        elif hypo == 'A':
            return 0.5
        else:
            return 1

In [10]:
hypos = 'ABC'
pmf = Monty(hypos)

In [11]:
data = 'B'
pmf.Update(data)

In [12]:
for hypo, prob in pmf.Items():
    print(hypo, prob)

A 0.3333333333333333
B 0.0
C 0.6666666666666666


### 2.5 - Encapsulando o framework

In [8]:
class Suite(Pmf):
    
    def __init__(self, hypos=tuple()):
        super().__init__(self)
        for hypo in hypos:
            self.Set(hypo, 1)
        self.Normalize()
    
    def Update(self, data):
        for hypo in self.Values():
            like = self.Likelihood(data, hypo)
            self.Mult(hypo, like)
        self.Normalize()
    
    def Likelihood(self, data, hypo):
        
        raise UnimplementedMethodException()
    
    def Print(self):
        for hypo, prob in self.Items():
            print(hypo, prob)

In [15]:
class Monty(Suite):
    
    def Likelihood(self, data, hypo):
        if hypo == data:
            return 0
        elif hypo == 'A':
            return 0.5
        else:
            return 1

In [17]:
suite = Monty('ABC')
suite.Update('B')
suite.Print()

A 0.3333333333333333
B 0.0
C 0.6666666666666666


### 2.6 - O problema do M&M

In [9]:
class M_and_M(Suite):
    mix94 = dict(brown=30, yellow=20, red=20, green=10, orange=10, tan=10)
    mix96 = dict(brown=13, yellow=14, red=13, green=20, orange=16, blue=24)
    
    hypoA = dict(bag1=mix94, bag2=mix96)
    hypoB = dict(bag1=mix96, bag2=mix94)
    
    hypotheses = dict(A=hypoA, B=hypoB)
    
    def Likelihood(self, data, hypo):
        bag, color = data
        mix = self.hypotheses[hypo][bag]
        like = mix[color]
        return like


In [10]:
suite = M_and_M('AB')
suite.Update(('bag1', 'yellow'))
suite.Update(('bag2', 'green'))

suite.Print()

A 0.7407407407407407
B 0.2592592592592592


### 2.7 - Discussões

### 2.8 - Exercícios